In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [4]:
df =pd.read_csv('HR_comma_sep.csv')

pd.options.display.max_columns = None

FileNotFoundError: [Errno 2] No such file or directory: 'HR_comma_sep.csv'

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.sample(10)

Variables transformations
To plot some statistical graphics and for better understanding, we make some transformations in the variables:

sales: Rename to department
salary: Convert the type of the variable from categorical to numerical.

In [ ]:

# RENAME column sale to department
df.rename(columns={'sales': 'department'}, inplace = True)

# Convert salary variable type to numeric
df['salary'] = df['salary'].map({'low':1, 'medium':2, 'high':3})



4. Descripitve Analysis
The descripitve Analysis is used to simplify and summarize the mainly characteristics of the dataset. In other words, show what kind of information the dataset has. The Pandas method describe generates a descriptive statistics that summarize the central tendency, dispersion and shape of the dataset. By using this method in Human Resource dataset important insights is possible to see:

That approximately 24% os the employees left the company.
The satisfaction level is around 62% and performance is around 72%.
Employees work in average on 4 projects with 200 hours worked per month.

In [ ]:
df.describe()

### 4.1 How many employees works in each department?
Depending on how many employees work in each department, you can learn more about the type of company segment.

In [ ]:
print(df['department'].value_counts())

### 4.2 How many employees per salary range?
The employees salary is divided in Low (1), Medium (2) and High (3), distributed as follows:

In [ ]:
print(df['salary'].value_counts())

### how many employees per salary range and department?

In [ ]:
table = df.pivot_table(values="satisfaction_level", index="department", columns="salary",aggfunc=np.count_nonzero)
table

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns 
sns.set()

In [ ]:
f, axes = plt.subplots(2,2, figsize=(10,10), sharex=True)

plt.subplots_adjust(wspace=0.5)# adjust the space between the plots

sns.despine(left=True)

# plot a boxplot of satisfaction_level to see if there is outliers
sns.boxplot(x='satisfaction_level', y='salary' ,data=df ,orient='v',ax=axes[0,0])

# plot a boxplot of last_evaluation to see if there is outliers
sns.boxplot( x= 'last_evaluation', y='salary' , data=df ,orient='v',ax=axes[0,1])

# plot a boxplot of number_project to see if there is outliers
sns.boxplot( x= 'number_project', y='salary', data=df,orient='v', ax=axes[1,0])

# plot a boxplot of average_montly_hours to see if there is outliers
sns.boxplot( x= 'average_montly_hours',y='salary'  ,data=df, ax=axes[1,1]);

#Put a ; at the end of the last line to suppress the printing of output 

In [ ]:
sns.boxplot('time_spend_company',  data=df, orient='v')

In [ ]:
sns.boxplot(y='satisfaction_level',  data=df, orient='v')

### 4.5 Correlation Analysis
The correlation is a very useful statitiscal analysis that describes the degree of relationship between two variables. Let´s see the table below and the heat map to see what relationship are in the data.

In the heat map is possible to see:

Negative correlation of (-0.39) between satisfaction_level and the employees that left the company.
The highest positive correlation is between number of projects and average monthly hours (0.42).
Last_evaluation is high correlated to number_project(0.35)and average_monthly_hours(0.34).
Work_accident have a low negative correlation(-0.15)and salary (-0.16) with employees that left.

In [ ]:
corr = df.corr()
corr

In [ ]:
sns.set(style='white')

mask = np.zeros_like(corr, dtype=np.bool)

mask[np.triu_indices_from(mask)] = True

# Inserir a figura
f, ax = plt.subplots(figsize=(13,8))

cmap = sns.diverging_palette(10,220, as_cmap=True)

#Desenhar o heatmap com a máscara
ax = sns.heatmap(corr, mask=mask, cmap=cmap, vmax= .5, annot=True, annot_kws= {'size':11}, square=True, xticklabels=True, yticklabels=True, linewidths=.5, 
           cbar_kws={'shrink': .5}, ax=ax)
ax.set_title('Correlation between variables', fontsize=20);

### 5. Hypothesis
Now let's extract some more informations and testing some hypothesis

### 5.1 How many employees left the company?

In [ ]:
print(df['left'].value_counts()[1],"employees left the company")

In [ ]:
# The plot show the amount o employees that stayed and left the company.
plt.figure(figsize=(4,5))
ax = sns.countplot(df.left)
total = float(len(df))
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format(height/total),
            ha="center") 
plt.title('Stayed or Left', fontsize=14);

### First Hypothesis
The first hypothesis is that salary is the reason why the employees left the company. Let's see if is this correct.

In [ ]:
j = sns.factorplot(x='salary', y='left', kind='bar', data=df)
plt.title('Employees that left by salary level', fontsize=14)
j.set_xticklabels(['High', 'Medium', 'Low']);


In the graphic Salaries by department is possible to see the distribuition of the salaries by department.

Most of the employees of the sales department have low or medium salaries, this may be due that in some companies the sales commission is paid separately.
Technical department is in the second place where most of the employees receives low and medium salaries.

In [ ]:
h = sns.factorplot(x = 'salary', hue='department', kind ='count', size = 5,aspect=1.5, data=df, palette='Set1' )
plt.title("Salaries by department", fontsize=14)
h.set_xticklabels(['High', 'Medium', 'Low']);


In the graphic(Salary Comparison):

The manangement department has the biggest difference between the salary of the employees who stayed and those that left.
It's not possible to see a huge difference in other departments.
The first hypothesis looks very weak to be the main reason why the employees left the company.

In [ ]:
sns.set()
plt.figure(figsize=(10,5))
sns.barplot(x='department', y='salary', hue='left', data=df)
plt.title('Salary Comparison', fontsize=14);

### Second Hypothesis
It is a dangerous job?

The second hypothesis is: employees leave the company because work is not safe.

In [ ]:
sns.factorplot(x='Work_accident', y='left', kind='bar', data=df)
plt.title('Employees that had work accident', fontsize=14);


About 14% of the employees had a work accident, although of the high number only of accidents only 169 employees data left the company had work a accident. Then this hypothesis is discarded.

In [ ]:

print(df.Work_accident.sum())
print(df.Work_accident.mean())
print((df[df['left']==1]['Work_accident']).sum())

### Third Hypothesis
Is this company a good place to grow professionally?

In [ ]:
sns.factorplot(x='promotion_last_5years', y='left', kind='bar', data=df)
plt.title('Employees who have been promoted in the last 5 years', fontsize=14);


In the last five years only 319 employees had promotion, this is equivalent to 2% of all employees. This may be a problem because if it is difficult to get promoted many employees become unmotivated and start looking for a new job.

In [ ]:
print(df.promotion_last_5years.sum())
print(df.promotion_last_5years.mean())


Years in the company

In the graphic 'Years in the company' we can identify an important characteristic.

Employees with 7 or more years didn't left, maybe because with the passing of the years they are more confortable and not so interested in look for a new challenge in other company.
The problem starts when the employees have more than 3 years and get worst when they achieve 5 years.
It is too early to say that the difficult to get promoted is the main reason for the leaving of the employees, but more research is needed.

In [ ]:

plt.figure(figsize =(7,5))
bins = np.linspace(1.0, 11,10)
plt.hist(df[df['left']==1]['time_spend_company'], bins=bins, alpha=1, label='Employees Left')
plt.hist(df[df['left']==0]['time_spend_company'], bins=bins, alpha = 0.5, label = 'Employee Stayed')
plt.grid(axis='x')
plt.xticks(np.arange(2,11))
plt.xlabel('time_spend_company')
plt.title('Years in the company', fontsize=14)
plt.legend(loc='best');

Performance Analysis
There are 2 distincts groups of employees. A group with poor performance and other with high performance employees. It's natural that employees that don't work well leave the company, but the main problem is that the high performance employees is leaving too and it's necessary to understand why.

In [ ]:
plt.figure(figsize =(7,7))
bins = np.linspace(0.305, 1.0001, 14)
plt.hist(df[df['left']==1]['last_evaluation'], bins=bins, alpha=1, label='Employees Left')
plt.hist(df[df['left']==0]['last_evaluation'], bins=bins, alpha = 0.5, label = 'Employee Stayed')
plt.title('Employees Performance', fontsize=14)
plt.xlabel('last_evaluation')
plt.legend(loc='best');

It is possible to see that 98% of employees with few projects that left also have poor performance.

And 95% of the employees with 5 or more projects that left the company had the highest performance.

3 or 4 are the best number of projects.

In [ ]:
poor_performance_left = df[(df.last_evaluation <= 0.62) & (df.number_project == 2) & (df.left == 1)]
print('poor_performance_left:',len(poor_performance_left))

poor_performance_stayed = df[(df.last_evaluation > 0.62) & (df.number_project == 2) & (df.left == 1)]
print('poor_performance_stayed:',len(poor_performance_stayed))

print('\n')

high_performance_left= df[(df.last_evaluation <= 0.62) & (df.number_project >=5) & (df.left == 1)]
high_performance_stayed= df[(df.last_evaluation > 0.8) & (df.number_project >=5) & (df.left == 0)]
print('high_performance_left:',len(high_performance_left))
print('high_performance_stayed', len(high_performance_stayed))

plt.figure(figsize =(7,5))
bins = np.linspace(1.5,7.5, 7)
plt.hist(df[df['left']==1]['number_project'], bins=bins, alpha=1, label='Employees Left')
plt.hist(df[df['left']==0]['number_project'], bins=bins, alpha = 0.5, label = 'Employee Stayed')
plt.title('Number of projects', fontsize=14)
plt.xlabel('number_ projects')
plt.legend(loc='best');

Working hours
Again, there are 2 groups of employees. A group that works fewer hours and another that works more hours compared to the average hours worked.

In [ ]:
plt.figure(figsize =(7,5))
bins = np.linspace(80,315, 15)
plt.hist(df[df['left']==1]['average_montly_hours'], bins=bins, alpha=1, label='Employees Left')
plt.hist(df[df['left']==0]['average_montly_hours'], bins=bins, alpha = 0.5, label = 'Employee Stayed')
plt.title('Working Hours', fontsize=14)
plt.xlabel('average_montly_hours')
plt.xlim((70,365))
plt.legend(loc='best');

Clearly is possible to see that the employees with 6 projects or more, work on average 20% more hours.

In [ ]:
groupby_number_projects = df.groupby('number_project').mean()
groupby_number_projects = groupby_number_projects['average_montly_hours']
print(groupby_number_projects)
plt.figure(figsize=(7,5))
groupby_number_projects.plot();

With the information above the employees that left the company are grouped as:

Employees with 2 projects and worked less than the average of the company.
Employees with 5 or more projects that worked at least 20% more than the average.

In [ ]:
work_less_hours_left = df[(df.average_montly_hours < 200) & (df.number_project == 2) & (df.left == 1)]
print('work_less_hours_left:',len(work_less_hours_left))

work_more_hours_left = df[(df.average_montly_hours > 240) & (df.number_project >=5 ) & (df.left == 1)]
print('work_more_hours_left:',len(work_more_hours_left))

#<p><font color="red">Aqui você fala sobre a relação entre horas de trabalho e quantidade de projetos, mas isso não é exibido no gráfico</font></p>


Satisfaction Level
It is possible to see 3 interesting peaks in the satisfaction levels of the employees that left the company.

We have a peak of employees who are totally disappointed.
Another peak at 0.4, representing another group with the satisfaction level below the average.
And another amount in the range 0.7 and 0.9, with employees that left, although the high satisfaction.

In [ ]:
plt.figure(figsize =(7,5))
bins = np.linspace(0.006,1.000, 15)
plt.hist(df[df['left']==1]['satisfaction_level'], bins=bins, alpha=1, label='Employees Left')
plt.hist(df[df['left']==0]['satisfaction_level'], bins=bins, alpha = 0.5, label = 'Employee Stayed')
plt.title('Employees Satisfaction', fontsize=14)
plt.xlabel('satisfaction_level')
plt.xlim((0,1.05))
plt.legend(loc='best');

### Average satisfaction for years in the company

In [ ]:
groupby_time_spend = df.groupby('time_spend_company').mean()
groupby_time_spend['satisfaction_level']


When the employees becames unsatisfied
In next results it is clear the drop in satisfaction when employees are working on 6 or more projects.

In [ ]:
sns.set()
sns.set_context("talk")
ax = sns.factorplot(x="number_project", y="satisfaction_level", col="time_spend_company",col_wrap=4, size=3, color='blue',sharex=False, data=df)
ax.set_xlabels('Number of Projects');


Let´s see why the most valuable employees tend to leave.

From the employees that left with high performance, 4 or more years in the company and working on 5 or more project had:

Low satisfaction level,
Worked more hours,
Haven´t been promoted in the last five years.

In [ ]:
func_living = df[(df.last_evaluation >= 0.70) | (df.time_spend_company >=4) | (df.number_project >= 5)]

corr2 = func_living.corr()

sns.set(style='white')

mask = np.zeros_like(corr2, dtype=np.bool)

mask[np.triu_indices_from(mask)] = True

# Insert the graphic
f, ax = plt.subplots(figsize=(13,8))

cmap = sns.diverging_palette(10,220, as_cmap=True)

#Draw heat map mask
ax = sns.heatmap(corr2, mask=mask, cmap=cmap, vmax= .5, annot=True, annot_kws= {'size':11}, square=True, xticklabels=True, yticklabels=True, linewidths=.5, 
           cbar_kws={'shrink': .5}, ax=ax)
ax.set_title('Correlation: Why Valuable Employees Tend to Leave', fontsize=20);

### Summary of the Exploratory Data Analysis
It is a relatively young company, on average, employees have 3 or 4 years in the company and the oldest employees are working 10 years.
The biggest difference in the salary from who stayed and those who left, was found in the managemnet department, in the others departments although the salaries of who stayed be higher in average, it is not a big difference.
The number of employees that had a work accident is about 14%, of which only 169 employees left the company, so don't seem to have a correlation with the employees leaving.
In five years only 2% of the employees were promoted. Is possible that many employees get unmotivated and start planning to leave.
Employees with 7 or longer in the company didn't left. Employees with 5 years have more chances to leaving.
There are 2 distincts groups of employees performance that left. A group with poor performance with 2 projects and others with high performance with 5 or more projects. It is not necessary retain all the employees, the focus is on keeping employees with high performance.
The employees with 4 years in the company have the lowest average satisfaction level of all the company with (0.47).
The satisfaction drops when the employees are working in 5 or more projects. A number of 3 or 4 projects seems to be ideal independent of the time spend in the company.
The employees with 5 or more projects that left also worked at least 20% more hours than the average of the company.
The satisfaction level of the employees that left is grouped in totally disappointed, below the average satisfaction and satisfied.